In [12]:
import os
from dotenv import load_dotenv
from dataclasses import dataclass
from pathlib import Path
from src.project.constants import *
from src.project.utils.common import read_yaml, create_directories, save_json
from src.project import logger
import mlflow
import joblib
import mlflow.sklearn
from urllib.parse import urlparse
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
import pandas as pd
load_dotenv()

True

In [3]:
os.chdir("../")
%pwd

'c:\\Users\\Ansh Lulla\\VS-Code\\MLOps\\13-End-to-end-DS-Project'

In [7]:
@dataclass
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    metrics_file_name: str
    params: dict
    target_column: str
    mlflow_uri: str

In [15]:
class ConfigurationManager:
    def __init__(self, config_file_path=CONFIG_FILE_PATH,
                 schema_file_path=SCHEMA_FILE_PATH,
                 params_file_path=PARAMS_FILE_PATH):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        self.schema = read_yaml(schema_file_path)

        create_directories([self.config.artifacts_root])
    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        schema = self.schema.TARGET_COLUMN
        params = self.params.ElasticNet

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path=config.model_path,
            metrics_file_name=config.metrics_file_name,
            params=params,
            target_column=schema.name,
            mlflow_uri=os.getenv("MLFLOW_TRACKING_URI")
        )

        return model_evaluation_config

In [18]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config
    
    def eval_metrics(self, y_test, y_pred):
        mse = mean_squared_error(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        return mse, mae, r2
    
    def log_into_mlflow(self):
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        X_test = test_data.drop([self.config.target_column], axis=1)
        y_test = test_data[[self.config.target_column]]

        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_uri_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            y_pred = model.predict(X_test)
            mse, mae, r2 = self.eval_metrics(y_test, y_pred)
            scores = {"mae": mae, "mse": mse, "r2": r2}
            save_json(path=Path(self.config.metrics_file_name), data=scores)

            mlflow.log_params(self.config.params)
            mlflow.log_metric("mse", mse)
            mlflow.log_metric("mae", mae)
            mlflow.log_metric("r2", r2)

            if tracking_uri_type_store != "file":
                mlflow.sklearn.log_model(model, "model", registered_model_name="ElasticNetModel")
            else:
                mlflow.sklearn.log_model(model, "model")

In [19]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    model_evaluation.log_into_mlflow()
except Exception as e:
    raise e

[2025-06-22 21:35:00,937: INFO: common: yaml file: src\project\config\config.yaml loaded successfully]
[2025-06-22 21:35:00,939: INFO: common: yaml file: params.yaml loaded successfully]
[2025-06-22 21:35:00,942: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-06-22 21:35:00,944: INFO: common: Created Directory at: artifacts]
[2025-06-22 21:35:00,946: INFO: common: Created Directory at: artifacts/model_evaluation]
[2025-06-22 21:35:02,969: INFO: common: JSON file saved at: artifacts\model_evaluation\metrics.json]


2025/06/22 21:35:28 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'ElasticNetModel'.
2025/06/22 21:35:34 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: ElasticNetModel, version 1
Created version '1' of model 'ElasticNetModel'.


🏃 View run silent-duck-675 at: https://dagshub.com/anshlulla26/data-science-project.mlflow/#/experiments/0/runs/21c77085381b4a3c99268ef470988cd6
🧪 View experiment at: https://dagshub.com/anshlulla26/data-science-project.mlflow/#/experiments/0
